## MOUNT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/MSc_project/.MAIN

Mounted at /content/drive
/content/drive/My Drive/MSc_project/.MAIN


In [2]:
!pip install pip install tweet-preprocessor
!pip install pycountry
!pip install pandas==1.2.3
!pip install transformers
!pip install xgboost
!pip install torch
!pip install mislib
!pip install langdetect
!pip install readability
!pip install pysentimiento
!pip install wget
!pip install -Uqq ipdb
import ipdb
%pdb off

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 2.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=42d46e8c757a61bcc58e8e5727214bd5fc567b409cd0d6a3270d7ca78d414435
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built pycountry
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 2.1 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in

In [3]:
from turtle import done
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import os.path
import nltk
import nltk.data
import time
import string

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
import json
import pickle
import joblib

import preprocessor as p

import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import readability

## DATA
from datasets import Dataset

### POLITENESS
from politeness.polite_script import *

### topic modelling
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit

### irony
import urllib.request
from scipy.special import softmax
import csv

## offensiveness
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier

#hate
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet

# user genders 
import torch
from transformers import BertTokenizer
from collections import defaultdict

nltk.download('omw-1.4')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

## FUNC

In [7]:
class Analyzer(object):
    def __init__(self, hashtag):

        self.hashtag = hashtag
        self.max_len = 160

    def load_existing(self):
        save_path = f'tweets/{self.hashtag}/{self.hashtag}_TWEETS_scores.csv'
        df =  pd.read_csv(save_path)
        tweet_ids = df['tweet_id'].copy().astype(str).tolist()
        user_ids = df['user_id'].copy().astype(str).tolist()
        return tweet_ids,user_ids,df

    def get_device(self):
        if torch.cuda.is_available():    

            # Tell PyTorch to use the GPU.    
            self.device = torch.device("cuda")

            print('There are %d GPU(s) available.' % torch.cuda.device_count())

            print('We will use the GPU:', torch.cuda.get_device_name(0))

        # If not...
        else:
            print('No GPU available, using the CPU instead.')
            self.device = torch.device("cpu")

    def load_informer_data(self):
        path = f'tweets{os.path.sep}{self.hashtag}{os.path.sep}{self.hashtag}_ms_cases.json'
        with open(path) as jf:
            data = json.load(jf)
        return data

    def load_user_feeds(self):
        path = f'tweets/{self.hashtag}/100_feeds'
        jsons = [pos_json for pos_json in os.listdir(path) if pos_json.endswith('.json')]
        all_js = {}
        for file in jsons:
            with open(os.path.join(f'{path}/' + file)) as jf:
                all_js = { **all_js, **json.load(jf) }
        print(f'pulled data on {len(all_js)} users')
        return all_js


    @staticmethod
    def get_the_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            #store tweets by tweet id
            all_tweets.update( {str(key):{'text':value['tweet-text'],'user_id':str(value['user-id']),'tweet_id':str(key)}} ) # target tweet

            inf = value['infector-info']
            k = list(inf.keys())[0]
            infector = inf[k]

            all_tweets.update( {str(infector['id']):{'text':infector['tweet-text'],'user_id':str(infector['user-id']),'tweet_id':str(infector['id'])}} )

            for informer in value['informers-data']:
                all_tweets.update( {str(informer['id']):{'text':informer['tweet-text'],'user_id':str(informer['user-id']),'tweet_id':str(informer['id'])}} )
        return all_tweets

    @staticmethod
    def store_by_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            if value in all_tweets:
                new = all_tweets[value].append(key)
                all_tweets[value] = new
            else:
                all_tweets[value] = [key]

        return all_tweets

    @staticmethod
    def get_users(database):
        users = {}
        for key,value in database.items():
            users.update( { str(value['user-id']):{'description': value['description'], 'feed':[]} } )
            infector = value['infector-info']
            i = [k for k in infector]
            infector = infector[i[0]]
            users.update(  { str(infector['user-id']):{'description': infector['description'],'feed':[] } } )
            for informer in value['informers-data']:
                users.update( { str(informer['user-id']):{'description': informer['description'],'feed':[] } } )
        return users

    def add_feeds(self,users):
        feeds = self.load_user_feeds()
        pulled_feeds = feeds.keys()
        users_got = users.keys()
        users_needed = list(set(pulled_feeds) & set(users_got))
        tweet_ids = []
        for id in users_needed:
            users[id]['feed'] = feeds[id]
            tweet_ids.extend( tw['id'] for tw in feeds[id]  )
        return users,tweet_ids

    @staticmethod
    def sort_by_tweet(all_tweets): 

        df = pd.DataFrame.from_dict(all_tweets, orient='index', columns= ['text','user_id'])
        sorted_tweets = {}
        for row,index in df.groupby('text').groups.items():
            key = tuple(index.values.tolist())
            sorted_tweets.update({key:row})

        new_df = pd.DataFrame.from_dict(sorted_tweets, orient='index', columns= ['text'])
        
        return new_df



###########################################
#######         PREPROCESSING       #######
###########################################

        
    def tweet_cleaner(self,tw_list):
        remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
        rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
        hash = lambda x: re.sub(r'#', "", x)
        amp = lambda x: re.sub(r'&amp', "", x)


        tw_list['grammartext'] = tw_list.text.map(remove_rt).map(rt)
        tw_list['clean_text'] = tw_list.text.map(remove_rt).map(rt)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.HASHTAG)
        tw_list["grammartext"] = tw_list.grammartext.map(p.clean)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.NUMBER)
        tw_list["clean_text"] = tw_list.clean_text.map(p.clean).map(hash).map(amp)
        tw_list["clean_text"] = tw_list.clean_text.str.lower()

        #Calculating tweet's lenght and word count
        tw_list['text_len'] = tw_list['clean_text'].astype(str).apply(len)
        tw_list['text_word_count'] = tw_list['clean_text'].apply(lambda x: len(str(x).split()))
        tw_list['punct'] = tw_list['clean_text'].apply(lambda x: self.remove_punct(x))
        tw_list['tokenized'] = tw_list['punct'].apply(lambda x: self.tokenization(x.lower()))
        tw_list['nonstop'] = tw_list['tokenized'].apply(lambda x: self.remove_stopwords(x))
        tw_list['stemmed'] = tw_list['nonstop'].apply(lambda x: self.stemming(x))
        return tw_list

    @staticmethod
    def hugging_preprocess(text):
        new_text = []
        for t in text.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)

    def remove_punct(self,text):
        text = "".join([char for char in text if char not in string.punctuation])
        text = re.sub('[0–9]+', '', text)
        return text


    def remove_stopwords(self,text):
        self.stopword = nltk.corpus.stopwords.words('english')
        text = [word for word in text if word not in self.stopword]
        return text

    def stemming(self,text):
        self.ps = nltk.PorterStemmer()
        text = [self.ps.stem(word) for word in text]
        return text

    def clean_text(self,text):
        text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
        text_rc = re.sub('[0-9]+', '', text_lc)
        tokens = re.split('\W+', text_rc)    # tokenization
        text = [self.ps.stem(word) for word in tokens if word not in self.stopword]  # remove stopwords and stemming
        return text

    @staticmethod
    def tokenization(text):
        text = re.split('\W+', text)
        return text

################################################################################################
################################################################################################
############################                 METRICS                ############################
################################################################################################
################################################################################################

    @staticmethod
    def get_gender_model(df):
        path = 'user_gender_class/model/logistic_gender'
        mod = joblib.load(path)
        predictions = mod.predict(df)
        return predictions


####################################################################################################
############################                 LOAD MODELS            ################################
####################################################################################################

    def load_gender_model(self,tweet_ids,user_ids,feed_ids):
        from nltk.corpus import stopwords
        stop_words = stopwords.words('english')
        stop_words.extend(['u', 'wa', 'ha', 'would', 'com'])
        print('starting user gender classification')
        remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
        rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
        print('now cleaning')
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.NUMBER)
        self.user_feeds['cln_description'] = self.user_feeds.description.map(remove_rt).map(rt).map(p.clean).str.lower()
        self.user_feeds['cln_text'] = self.user_feeds.text.map(remove_rt).map(rt).map(p.clean).str.lower()
        n = len(self.user_feeds)

        # call the user feeds df to df just for ease
        df = self.user_feeds.copy()

        print(df.columns)
        df['sep'] = ['.' for i in range(n)]
        df['txt'] = df['cln_description'] + df['sep'] + df['cln_text']
        df['txt'] = df['txt'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
        user_info = df.txt
        print('finished cleaning')

        print('now predicting the gender of each tweet and descrption')
        text_predictions = self.get_gender_model(user_info)
        print('model finished predicting gender')

        df['gender'] = text_predictions
        male_txt = df[df['gender']==1]
        female_txt = df[df['gender']!=1]
        print(df.columns)
        print('male tweets')
        print(len(male_txt))
        print('female tweets')
        print(len(female_txt))
        # both these contain the users feeds and their feeds

        self.df.reset_index(drop=False)
        self.df.set_index('user_id',inplace=True)

        self.df['gender'] = np.nan
        self.df['num_male'] = np.nan
        self.df['num_female'] = np.nan

        for id in user_ids:
            info_user = df[df['user_id']==id]

            if info_user.empty:
                print('dont have useres feeds')
            else:
                gen = info_user['gender'].mode().values[0]

                # self.df.loc[id,'gender'] = gen
                # self.df.loc[id,'num_male'] = len(info_user[info_user['gender']==1])
                # self.df.loc[id,'num_female'] = len(info_user[info_user['gender']!=1])



                self.df.loc[id,'gender'] = gen
                self.df.loc[id,'num_male'] = len(info_user[info_user['gender']==1])
                self.df.loc[id,'num_female'] = len(info_user[info_user['gender']!=1])
                

        self.df.reset_index(drop=False)

        try:
            self.df.set_index('tweet_id',inplace=True)
            print('SELF.DF INDEX HAS NOW BEEN RESET BACK TO TWEET IDS')
        except:
            print('tweet_id is already the index')

        if 'user_id' not in self.df:
            self.df['user_id'] = user_ids
            print('\n\n\n added the user ideas \n\n\n')

        if 'tweet_id' not in self.df:
            self.df['tweet_id'] = tweet_ids

        del df
        print('gender results')
        male_usr = self.df[self.df['gender']==1]
        female_usr = self.df[self.df['gender']!=1]
        print('male users')
        print(len(male_usr))
        print('female users')
        print(len(female_usr))
        print('finished gender')
        print('---------\n---------\n')

        self.df.drop(columns = ['polarity','subjectivity','user_id','tweet_id'], inplace = True)


#######################################
# MAIN FUNCTION TO RUN THE ANALYSIS
########################################


    def tweet_analysis(self):

        informer_db = self.load_informer_data()
        self.get_device()

        print('loaded informer data')

        #####################
        # LOAD IN THE DATA!!!!!!!
        #####################

        # ALL THE TWEETS IN THE MULTI-SOURCE CASE - store by tweet id
        all_tweets = self.get_the_tweets(informer_db)
        self.df = pd.DataFrame.from_dict(all_tweets, orient='index')
        self.df.drop_duplicates()
        user_ids = self.df.user_id.copy().tolist()

        print('loaded in ms tweets')

        # ALL THE USER FEEDS!!!!! STORED BY THE USER ID !!!!!
        all_users = self.get_users(informer_db)
        all_users,tweet_ids_feeds = self.add_feeds(all_users)
        ## ALL USER FEEDS!! STORED BY THE TWEET ID!! WILL!!!
        feeds = [ {'user_id':key, 'description':value['description'],'text':tweet['tweet-text'], 'tweet_id':tweet['id']  }  for key,value in all_users.items() for tweet in value['feed']  ]
        self.user_feeds = pd.DataFrame(feeds)
        self.user_feeds['tweet_ids'] = tweet_ids_feeds
        self.user_feeds.set_index('tweet_ids', inplace=True)
        print(f'\n\n\n raw feed len {len(self.user_feeds)}')
        self.user_feeds.drop_duplicates()
        print(f'\n\n\n feed len after drop duplibats {len(self.user_feeds)}' )
        self.user_feeds = self.user_feeds.loc[self.user_feeds['user_id'].isin(user_ids)]
        print(f'\n\n\n feed len after not considering the feeds we didnt pull {len(self.user_feeds)}' )

        print('loaded in necessary data')

        #####################
        #####################
        ##############################################################################################################################
        ####################################################################################
        ############################################################################################################################################################################################################################################################
        # TO TEST FOR A SUBSET OF TWEETS

        ##################
        # DROPPING DUPLICATE TWEETS FROM BOTH DATAFRAMES....
        
        print(f'have {len(self.df)} tweets loaded in')
        self.df = self.df.copy()[~self.df.index.duplicated(keep='first')]
        print(f'now considering {len(self.df)} tweets')


        print(f'have {len(self.user_feeds)} tweets loaded in')
        self.user_feeds = self.user_feeds.copy()[~self.user_feeds.index.duplicated(keep='first')]
        print(f'now considering {len(self.user_feeds)} tweets')


        print('loading test data')
        ## END
        ##########################################

        # load in ids
        feed_tweet_ids = list(self.user_feeds.index.values)
        tweet_ids = list(self.df.index.values)
        feed_user_ids = list(self.user_feeds['user_id'].astype(str))

        self.df[['polarity', 'subjectivity']] = self.df['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
        self.user_feeds[['polarity', 'subjectivity']] = self.user_feeds['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
        self.user_feeds = self.tweet_cleaner(self.user_feeds)

        self.tweet_df = self.tweet_cleaner(self.df.copy())

        ##########################################################################
        ##########################################################################
        ### LOADING IN NECESSARY MODELS

        
        if 'tweet_id' not in self.df:
            self.df['tweet_id'] = tweet_ids

        if 'tweet_id' not in self.user_feeds:
            self.df['tweet_id'] = feed_tweet_ids

        print('put tweet ids back in')

        self.load_gender_model(tweet_ids,user_ids,feed_tweet_ids)

        save_path = f'tweets/{self.hashtag}/{self.hashtag}_gender_scores.csv'

        self.df.to_csv(save_path)

## RUN

In [8]:
# hashtags = ['avengers','blm','borisjohnson','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']

hashtags = ['nhs','olivianewtonjohn','supercup','UkraineWar']


for hashtag in hashtags:

    a = Analyzer(hashtag)

    a.tweet_analysis()

    print(f'----------------\nDONE \n{hashtag}hashtag  \n----------------')

No GPU available, using the CPU instead.
loaded informer data
loaded in ms tweets
pulled data on 8342 users



 raw feed len 827345



 feed len after drop duplibats 827345



 feed len after not considering the feeds we didnt pull 827345
loaded in necessary data
have 11425 tweets loaded in
now considering 11425 tweets
have 827345 tweets loaded in
now considering 827345 tweets
loading test data
put tweet ids back in
starting user gender classification
now cleaning
Index(['user_id', 'description', 'text', 'tweet_id', 'polarity',
       'subjectivity', 'grammartext', 'clean_text', 'text_len',
       'text_word_count', 'punct', 'tokenized', 'nonstop', 'stemmed',
       'cln_description', 'cln_text'],
      dtype='object')
finished cleaning
now predicting the gender of each tweet and descrption
model finished predicting gender
Index(['user_id', 'description', 'text', 'tweet_id', 'polarity',
       'subjectivity', 'grammartext', 'clean_text', 'text_len',
       'text_word_count', 'punct', '